# Read Me


# Introduction

The purpose of this assignment to build a seq2seq model that will self-align audio to phonemes using CTC loss

# How to Train
This notebook contains all the code necessary to train the model. To train, run all of the cells up the section ***Testing(Sections 1-4)***
starting from the first cell. Once the training is complete, you will want to check the output for the model with the best accuracy. Each model will be of the from model{epoch #}.txt, for example model40.txt had the best validation accuracy so that is what I chose.

# How to Test

## Classification
Run all the cells under in ***Section 5***. Note this will submit to kaggle.

# Model Description

See notebook for more details:
I used one resnet block with batchnorm, 4 bidrectional layers of an LSTM, followed by two linear layers to output.








# 1 Setup

## 1.1 Google Drive - Kaggle

In [ ]:
# Google drive setup
# from google.colab import drive

# gdrive.mount("/content/gdrive", force_remount=True)

In [ ]:
import json

api_token = {"username":"bustin1","key":"914b2f1eda974e0940999f8187a85285"}

!mkdir .kaggle
!mkdir ~/.kaggle

with open('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle --version

     |████████████████████████████████| 58 kB 2.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=630be5ef9836925a27e83206c38e1388dcd98af10ac6b5efc258b4ca3c9af3fe
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Kaggle API 1.5.12


## 1.2 Kaggle Data Download

In [ ]:
# download data
!kaggle competitions download -c 11785-fall2021-hw3p2

 99% 2.33G/2.35G [00:10<00:00, 159MB/s]
100% 2.35G/2.35G [00:11<00:00, 213MB/s]


In [ ]:
!mkdir data

!unzip -qo './11785-fall2021-hw3p2.zip' -d data 

In [ ]:
!ls data/

HW3P2_Data


## 1.3 Library Installations

Install [ctcdecode](https://github.com/parlance/ctcdecode)

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 12.01 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14051, done.        
remote: Counting objects: 100% (364/364), done.        
remote: Compressing objects: 100% (296/296), done.        
remote: Total 14051 (delta 10

Install [levenshtein distance calculation library](https://github.com/ztane/python-Levenshtein) 

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149878 sha256=be80a99b877444129909a3e08ea0bd1df288afdd2a28320ec2501be4560a9b8d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


## 1.4 Libraries & Setup

In [ ]:
import os
import sys
import time

from Levenshtein import distance as lev

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pdb
import gc
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold

import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 2 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  2  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


# 2 Data Loading

## 2.1 Load Data

In [ ]:
# load training and dev data
train_data = np.load('data/HW3P2_Data/train.npy', allow_pickle=True)
train_labels = np.load('data/HW3P2_Data/train_labels.npy', allow_pickle=True)

dev_data = np.load('data/HW3P2_Data/dev.npy', allow_pickle=True)
dev_labels = np.load('data/HW3P2_Data/dev_labels.npy', allow_pickle=True)

# combine both into one big data for k-fold CV
all_data = np.append(train_data, dev_data)
all_data_labels = np.append(train_labels, dev_labels)

# load test data
test_data = np.load('data/HW3P2_Data/test.npy', allow_pickle=True)

In [ ]:
print(f'Train data: {train_data.shape}')
print(f'Train labels {train_labels.shape}')

print(f'Dev data: {dev_data.shape}')
print(f'Dev labels {dev_labels.shape}')

print(f'Test data: {test_data.shape}')
print(f'Train Labels: {train_labels[0]}')

print(f"All data:{all_data.shape}")
print(f"All data Labels: {all_data_labels.shape}")


Train data: (14542,)
Train labels (14542,)
Dev data: (2200,)
Dev labels (2200,)
Test data: (2561,)
Train Labels: [1, 12, 5, 23, 4, 33, 14, 1, 32, 35, 30, 5, 37, 22, 38, 13, 23, 19, 26, 12, 5, 33, 13, 24, 29, 5, 31, 33, 38, 19, 25, 12, 5, 38, 15, 37, 40, 9, 19, 22, 15, 24, 33, 36, 37, 8, 23, 5, 25, 33, 1, 5, 23, 7, 11, 12, 5, 24, 33, 19, 30, 8, 40, 33, 36, 5, 31, 14, 33, 5, 25, 29, 19, 10, 1, 12, 5, 33, 18, 20, 24, 8, 33, 9, 20, 30, 20, 37, 13, 25, 21, 11, 1, 3, 25, 12, 20, 1, 19, 24, 29, 6, 30, 10, 5, 25, 5, 33, 1, 3, 25, 11, 30, 15, 5, 1, 4, 25, 11, 9, 19, 31, 8, 11, 40, 1, 19, 33, 38, 35, 11, 5, 16, 6, 30, 11, 18, 19, 24, 31, 5, 24, 30, 13, 22, 30, 20, 15, 32, 5, 25, 11, 14, 19, 26, 12, 5, 23, 6, 26, 11, 15, 1]
All data:(16742,)
All data Labels: (16742,)


## 2.2 Custom Dataset Classes

In [ ]:
# Define dataset class
class MyDataSet(Dataset):
  # load the dataset
  def __init__(self, x, y):
    self.X = x
    self.Y = y

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.Y)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    y = torch.LongTensor(self.Y[index])

    return x, y

  def collate_fn(batch):
    (xx, yy) = zip(*batch) # (batch, x.shape)
    
    x_lens = torch.tensor([len(x) for x in xx])
    y_lens = torch.tensor([len(y) for y in yy])

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0) # BxLxC
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=0)

    return xx_pad.to(device), yy_pad.to(device), x_lens, y_lens

In [ ]:
# Define dataset class
class TestDataSet(Dataset):
  # load the dataset
  # TODO: replace x and y with dataset path and load data from here -> more efficient
  def __init__(self, x):
    self.X = x

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.X) 

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    return x

  def collate_fn(batch):
    xx = batch
    x_lens = torch.tensor([len(x) for x in xx])
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    return xx_pad.to(device), x_lens


## 2.3 Data Loaders

In [ ]:
batch_size = 64

# training data
train = MyDataSet(train_data, train_labels)
train_args = {'shuffle': True, 'batch_size':batch_size, 'collate_fn': MyDataSet.collate_fn}
train_loader = DataLoader(train, **train_args)

# validation data
dev = MyDataSet(dev_data, dev_labels)
dev_args = {'shuffle': False, 'batch_size':batch_size, 'collate_fn':MyDataSet.collate_fn}
dev_loader = DataLoader(dev, **dev_args)

# test data
test = TestDataSet(test_data)
test_args = {'shuffle': False, 'batch_size':batch_size, 'collate_fn':TestDataSet.collate_fn}
test_loader = DataLoader(test, **test_args)

# training data + validation data
all_dataset = MyDataSet(all_data, all_data_labels)


# 3 Model Building

## 3.1 Model Creation

In [ ]:
class RNNModel(nn.Module):
  def __init__(self, input_size, hs, nl, output_size):
    super(RNNModel, self).__init__()
    self.embed = nn.Sequential (
        nn.Conv1d(input_size, 128, kernel_size=3, stride=1, bias=False, padding=1), # acts on the frequencies
        nn.BatchNorm1d(128),
        nn.ELU(),
        nn.Conv1d(128, 256, kernel_size=3, stride=1, bias=False, padding=1), # acts on the frequencies
        nn.BatchNorm1d(256),
        nn.ELU(),
    )
    self.rnn = nn.LSTM(input_size=256, hidden_size=hs, num_layers=nl, bidirectional=True)
    self.drop = nn.Dropout(p=.2)
    self.fc = nn.Sequential(
        nn.Linear(hs*2, hs), # *2 for bidirectional
        self.drop,
        nn.Linear(hs, output_size)
    )
    self.log_sm = nn.LogSoftmax(dim=2)

  def forward(self, x, length):
    x = x.permute(0, 2, 1) # BxCxL
    x = self.embed(x)
    x = x.permute(2, 0, 1) # LxBxC
    x = pack_padded_sequence(x, length, batch_first=False, enforce_sorted=False)
    x, _ = self.rnn(x)
    x, _ = pad_packed_sequence(x, batch_first=False)
    x = x.permute(1, 0, 2) # BxLxC
    x = self.fc(x)
    x = self.log_sm(x)
    x = x.permute(1,0,2) # LxBxC
    return x

# class RNNModel(nn.Module): 
#   def __init__(self, input_size, hs, nl, output_size): 
#     super(RNNModel, self).__init__() 
#     self.embed = nn.Conv1d(input_size, 256, kernel_size=3, stride=1, bias=False, padding=1) # acts on the frequencies 
#     self.rnn = nn.LSTM(input_size=256, hidden_size=hs, num_layers=nl, bidirectional=True, dropout=.2) 
#     self.drop = nn.Dropout(p=.2) 
#     self.fc = nn.Sequential( 
#         self.drop, 
#         nn.Linear(2*hs, hs), # *2 for bidirectional 
#         self.drop, 
#         nn.Linear(hs, int(hs/2)), 
#         self.drop, 
#         nn.Linear(int(hs/2), output_size) 
#     ) 
#     self.log_sm = nn.LogSoftmax(dim=2) 
 
#   def forward(self, x, length): 
#     x = x.permute(0, 2, 1) # BxCxL 
#     x = self.embed(x) 
#     x = x.permute(2, 0, 1) # LxBxC 
#     x = pack_padded_sequence(x, length, batch_first=False, enforce_sorted=False) 
#     x, _ = self.rnn(x) 
#     x, _ = pad_packed_sequence(x, batch_first=False) 
#     x = x.permute(1, 0, 2) # BxLxC 
#     x = self.fc(x) 
#     x = self.log_sm(x) 
#     x = x.permute(1,0,2) # LxBxC 
#     return x 


# class RNNModel(nn.Module):
#   def __init__(self, input_size, hs, nl, output_size):
#     super(RNNModel, self).__init__()
#     self.embed = nn.Sequential (
#         nn.Conv1d(input_size, 128, kernel_size=3, stride=1, bias=False, padding=1), # acts on the frequencies
#         nn.BatchNorm1d(128),
#         nn.ELU(),
#         nn.Conv1d(128, 256, kernel_size=3, stride=1, bias=False, padding=1), # acts on the frequencies
#         nn.BatchNorm1d(256),
#         nn.ELU(),
#     )
#     self.rnn = nn.LSTM(input_size=256, hidden_size=hs, num_layers=nl, bidirectional=True)
#     self.drop = nn.Dropout(p=.2)
#     self.fc = nn.Sequential(
#         nn.Linear(hs*2, hs), # *2 for bidirectional
#         nn.SELU(),
#         self.drop,
#         nn.Linear(hs, output_size)
#     )
#     self.log_sm = nn.LogSoftmax(dim=2)

#   def forward(self, x, length):
#     x = x.permute(0, 2, 1) # BxCxL
#     x = self.embed(x)
#     x = x.permute(2, 0, 1) # LxBxC
#     x = pack_padded_sequence(x, length, batch_first=False, enforce_sorted=False)
#     x, _ = self.rnn(x)
#     x, _ = pad_packed_sequence(x, batch_first=False)
#     x = x.permute(1, 0, 2) # BxLxC
#     x = self.fc(x)
#     x = self.log_sm(x)
#     x = x.permute(1,0,2) # LxBxC
#     return x


## 3.2 Model Initialization

In [ ]:
# create model
input_size = 40
hidden_size = 256
num_layers = 4
output_size = 42

model = RNNModel(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('model4.txt'))
model = model.to(device)
print(model)

RNNModel(
  (embed): Sequential(
    (0): Conv1d(40, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
  )
  (rnn): LSTM(256, 256, num_layers=4, bidirectional=True)
  (drop): Dropout(p=0.2, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=256, out_features=42, bias=True)
  )
  (log_sm): LogSoftmax(dim=2)
)


# 4 Model Training

## 4.0 Set Hyperparameters

In [ ]:
# Hyperparams


criterion = nn.CTCLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=.002, weight_decay=5e-6) # CHANGE BACK TO .002
optimizer = torch.optim.Adam(model.parameters(), lr=.001, weight_decay=5e-6) # CHANGE BACK TO .002
# optimizer.load_state_dict(torch.load("./Adam20_plus_12_leven_11.txt"))
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=0, mode='min')
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)




## 4.1 Train Epoch

In [ ]:
# Train the model
def train_epoch(model, loader, criterion, optimizer):
  
  model.train()
  
  total_loss = batch_loss = num_correct = 0.0
  max_iter = len(loader) # should be global
  with tqdm(total=max_iter) as pbar:
    for i, (xx, yy, x_lens, y_lens) in enumerate(loader):
      # xx has shape (batch size, timestep (padded), frequency)
      # yy has shape (batch size, length of output (padded))
      # x_lens has shape (batch size, )
      # y_lens has shape (batch size, )

      output = model(xx, x_lens)
      loss = criterion(output, yy, x_lens, y_lens)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      batch_loss = loss.item()
      total_loss += batch_loss

      del xx
      del yy
      del loss
      del output
      del x_lens
      del y_lens
        
      pbar.set_description(f"(# {i}) batch loss: {batch_loss}")

      pbar.update(1)

      torch.cuda.empty_cache()

  avg_loss = total_loss / len(loader) # average batch loss
  # !nvidia-smi
  return avg_loss

## 4.2 CTC Decoding

### Phoneme list/map

In [ ]:
import sys
sys.path.append("data/HW3P2_Data")

N_PHONEMES = 41
PHONEME_LIST = [
    " ",
    "SIL",
    "SPN",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "H",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

PHONEME_MAP = [
    " ",
    ".", #SIL
    "!", #SPN
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

assert len(PHONEME_LIST) == len(PHONEME_MAP)
assert len(set(PHONEME_MAP)) == len(PHONEME_MAP)


### Create decoder

In [ ]:
from ctcdecode import CTCBeamDecoder

# TODO: Initialize decoder here
# In CTCBeamDecoder beam_width=1 (greedy search); beam_width>1 (beam search)
decoder = CTCBeamDecoder(
    PHONEME_MAP,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=15, # originally 100
    num_processes=2,
    blank_id=0,
    log_probs_input=True
)


## 4.3 Validate Epoch

In [ ]:
# Train the model
def validate_model(model, loader, criterion):

  avg_loss = 0.0
  running_dist = 0.0
  predictions = []

  with torch.no_grad():
    # model in validation mode 
    model.eval()
    total_loss = batch_loss = 0.0
    max_iter = len(loader) # should be global
    count = 0
    total_leven_dist = 0.0
    with tqdm(total=max_iter) as pbar:
      for i, (xx, yy, x_lens, y_lens) in enumerate(loader):
      
        # import pdb
        # pdb.set_trace()
        output = model(xx, x_lens)
        loss = criterion(output, yy, x_lens, y_lens)
        
        batch_loss = loss.item()
        total_loss += batch_loss

        del xx
        del loss

        output = output.permute(1, 0, 2) # BxLxC
        beam_results, _, _, out_lens = decoder.decode(output, x_lens)
        del output

        leven_dist = 0.0
        for i in range(beam_results.shape[0]):
          phoneme = "".join(PHONEME_MAP[n] for n in beam_results[i, 0, :out_lens[i][0]])
          phoneme_true = "".join(PHONEME_MAP[n] for n in yy[i, :y_lens[i]])
          leven_dist += lev(phoneme, phoneme_true)
          count += 1
        total_leven_dist += leven_dist

        pbar.update(1)
        pbar.set_description(f"(# {i}) batch loss: {batch_loss}, Leven dist: {leven_dist / beam_results.shape[0]}")
        
        del yy
        del beam_results
        del out_lens
        del y_lens
        del x_lens

        torch.cuda.empty_cache()
        

  avg_loss = total_loss / len(loader) # average batch loss
  avg_leven_dist = total_leven_dist / count
  # !nvidia-smi
  return avg_loss, avg_leven_dist

## 4.4 Run Epochs

In [ ]:
# Define number of epochs
epochs = 200

best_loss = float('inf')

print('Start...')
for epoch in range(1, epochs):
  print('Epoch: ', epoch)
  print(f"last lr: {optimizer.param_groups[0]['lr']}")

  train_loss = train_epoch(model, train_loader, criterion, optimizer)
  print(f'Avg Training Loss: {train_loss}')

  val_loss, leven_dist = validate_model(model, dev_loader, criterion)
  print(f'Avg Validation Loss: {val_loss}, Avg Leven: {leven_dist}')

  scheduler.step(leven_dist)
  

  # save the best model
  if leven_dist < best_leven_dist or epoch % 5 == 0:
    print('Best loss: {}, epoch: {}'.format(val_loss, epoch))
    torch.save(model.state_dict(), f'model{epoch}.txt')
    best_leven_dist = leven_dist

  print('='*40)
print('Done...')

Start...
Epoch:  1
last lr: 0.001


  0%|          | 0/228 [00:00<?, ?it/s]

Avg Training Loss: 0.08181635009353622


  0%|          | 0/35 [00:00<?, ?it/s]

Avg Validation Loss: 0.47176945209503174, Avg Leven: 11.063181818181818
Epoch:  2
last lr: 0.001


...

  0%|          | 0/228 [00:00<?, ?it/s]

Avg Training Loss: 0.0012204476760847396


  0%|          | 0/35 [00:00<?, ?it/s]

Avg Validation Loss: 0.857271991457258, Avg Leven: 10.610909090909091
Best loss: 0.857271991457258, epoch: 67
Epoch:  68
last lr: 4.355614296588023e-08


  0%|          | 0/228 [00:00<?, ?it/s]

## 4.4 Run on all dataset

In [ ]:
# # Define number of epochs
# epochs = 10

# best_loss = best_leven_dist = float('inf')
# all_train_loader = DataLoader(all_dataset, shuffle=True, batch_size=batch_size, collate_fn=MyDataSet.collate_fn)

# print('Start...')
# for epoch in range(1, epochs+1):
#   print('Epoch: ', epoch)

#   train_loss = train_epoch(model, all_train_loader, criterion, optimizer)
#   print(f'Avg Training Loss: {train_loss}')

#   # val_loss, leven_dist = validate_model(model, dev_loader, criterion)
#   # print(f'Avg Validation Loss: {val_loss}, Avg Leven: {leven_dist}')

#   scheduler.step()
#   print(f"last lr: {optimizer.param_groups[0]['lr']}")

#   # save the best model
#   torch.save(model.state_dict(), f'model{epoch}.txt')

#   print('='*40)
# print('Done...')

Start...
Epoch:  1


  0%|          | 0/262 [00:00<?, ?it/s]

Avg Training Loss: 0.0856193021110451
last lr: 0.000729
Epoch:  2


  0%|          | 0/262 [00:00<?, ?it/s]

Avg Training Loss: 0.06962280903893117
last lr: 0.0006561000000000001
Epoch:  3


...

Avg Training Loss: 0.04381181268405368
last lr: 0.0004782969
Epoch:  6


  0%|          | 0/262 [00:00<?, ?it/s]

Avg Training Loss: 0.03357062657996444
last lr: 0.00043046721
Epoch:  7


  0%|          | 0/262 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## 4.4 Run epochs (K-fold CV)

In [ ]:
# # MAKE SURE TO RERUN DATALOADERS IF YOU DO K-FOLD CV
# epochs = 200
# k = 10

# best_loss = float('inf')
# best_leven_dist = float('inf')
# splits=KFold(n_splits=k,shuffle=True)
# BS = 1

# print('Start...')
# for epoch in range(1, epochs//BS+1):
#   print('Epoch: ', epoch)
#   total_leven_dist = total_train_loss = total_val_loss = 0.0
#   for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(all_dataset)))):

#     print(f'Fold {fold+1}/{k}')
#     train_sampler = SubsetRandomSampler(train_idx)
#     dev_sampler = SubsetRandomSampler(val_idx)
#     local_train_loader = DataLoader(all_dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=MyDataSet.collate_fn)
#     local_dev_loader = DataLoader(all_dataset, batch_size=batch_size, sampler=dev_sampler, collate_fn=MyDataSet.collate_fn)
    
#     for i in range(BS):
#       train_loss = train_epoch(model, local_train_loader, criterion, optimizer)
#       print(f'({BS*epoch+i}) Avg Training Loss: {train_loss}', end='\t')

#       val_loss, leven_dist = validate_model(model, local_dev_loader, criterion)
#       print(f'Avg Validation Loss: {val_loss}, Avg Leven: {leven_dist}')

#       total_leven_dist += leven_dist
#       total_val_loss += val_loss
#       total_train_loss += train_loss

#     leven_dist = total_leven_dist/(i+1)/BS
#     scheduler.step(leven_dist)

#   print(f"last lr: {optimizer.param_groups[0]['lr']}\t Training loss: {total_train_loss/k/BS}\t Val Loss: {total_val_loss/k/BS}\t Leven Dist: {leven_dist}")

#   # save the best model
#   if leven_dist < best_leven_dist:
#     print('Best Leven Dist: {}, epoch: {}'.format(best_leven_dist, epoch))
#     torch.save(model.state_dict(), f'model{epoch}.txt')
#     best_leven_dist = leven_dist

#   print('='*40)
# print('Done...')

Start...
Epoch:  1
Fold 1/10


  0%|          | 0/236 [00:00<?, ?it/s]

(1) Avg Training Loss: 0.2897292567511736	

  0%|          | 0/27 [00:00<?, ?it/s]

Avg Validation Loss: 0.2845430440372891, Avg Leven: 10.61910447761194
Fold 2/10


...

(2) Avg Training Loss: 0.04824301139530489	

  0%|          | 0/27 [00:00<?, ?it/s]

Avg Validation Loss: 0.04348345680369271, Avg Leven: 1.9050179211469533
Fold 7/10


  0%|          | 0/236 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi


# 5 Test Data

## 5.1 test model function

In [ ]:
# Test the model
def test_model(model, test_loader):

  predictions = []

  with torch.no_grad():
    # model in validation mode 
    model.eval()
    max_iter = len(test_loader)
    with tqdm(total=max_iter) as pbar:
      for i, (xx, x_lens) in enumerate(test_loader):
      
        output = model(xx, x_lens)

        del xx

        output = output.permute(1, 0, 2) # BxLxC
        beam_results, _, _, out_lens = decoder.decode(output, x_lens)
        del output

        for i in range(beam_results.shape[0]):
          phoneme = "".join(PHONEME_MAP[n] for n in beam_results[i, 0, :out_lens[i][0]])
          predictions.append(phoneme)

        pbar.update(1)
        
        del beam_results
        del out_lens

        torch.cuda.empty_cache()
        
  return predictions

## 5.2 Make Predictions

In [ ]:
predictions = test_model(model, test_loader)


  0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
print(len(predictions))
print(predictions[0])
# .sedDhpRAktikhlIdiT.ydwntbilIv?UNOlhbitmoRhbwtWilHaRdhnzbREvrIDenHIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wh.HWytWenHIso.WiRHIWhzHIdidhnROndhWEoRflhNkwt.
# .sedDhpRAktikhlIdiT.ydwntbilIv?UNOhbitmoRhbwtWilHaRdhnzbREvrIDhnHIdidbIfoR.WInUshmTiNhbwtHizmAnrz.Wht.HWytWenHIso.WiRIWhzHIdidhntRhnhvWEoRflhNkw.
# .sedDhpRAktikhlItiT.ydwntbilIv?UnOhvbitmoRhbwtWilHaRthnsbREvRIDenHIdidbIfoR.WInUshmTiNhbwtHizmAnrz.Wht.HWy.WinHIso.WeRIWhzHIdinhntRhnhWEoRfoNgAt.
# .sedDhpRAkhkhlIdiT.ydOnpilIv?nOhbitmoRhbwtWilHaRdhnzbREvrIDenHIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wh.WyWinHIso.WiRHIWhzHIdidntRhnhWEoRflaNkw.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOhbitmoRhbwtWilHaRdhnsbREvrRIDenHIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.HWh.HWy.WinHIso.WeRHIWhzHIdidtRhnhWEoRflAgO.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOhbitmoRhbwtWilHaRdhnsbREveRIDenHIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wh.HWy.WinHIso.WeRHIWhzHIdidtRhnhWEoRflANkAp.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOhbitmoRhbwtWilHaRdhnsbREvuRIDenIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wht.HWyWinHIso.WeRHIWhzHIdidntRhnhWEoRfloNkhp.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOlhbitmoRhbwtWilHaRdhnzbREvRIDenHIdidbIfoR.WInUshmTiNhbwtHizmAnrz.Wh.HWy.WinHIso.WeRHIWozHIdidntRhnhWEoRflhNkh.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOlhbitmoRhbwtWilHaRdhnsbREvrIDenIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wht.HWylWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOlhbitmoRhbwtWilHaRdhnsbREvrIDenIdidbIfoR.WInUzshmTiNhbwtHizmAnrz.Wht.HWylWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOlhbitmoRhbwtWilHaRdhnsbREvrIDenHIdidbIfloR.WInUzshmTiNhbwtHizmAnrz.Wht.HWyWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOlhbitmoRhbwtWilHaRdhnsbREvrIDenHIdidbIfloR.WInUzshmTiNhbwtHizmAnrz.Wht.HWyWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.
# .sedDhpRAkthkhlIdiT.ydOntbilIv?UnOhbitmoRhbwtWilHaRdhnsbREvuRIDenHIdidbIfloR.WInUshmTiNhbwtHizmAnrz.Wht.HWylWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.




2561
.sedDhpRAkthkhlIdiT.ydOntbilIv?UnOhbitmoRhbwtWilHaRdhnsbREvuRIDenHIdidbIfloR.WInUshmTiNhbwtHizmAnrz.Wht.HWylWinHIso.WeRHIWhzHIdidhntRhnhWEoRflANkA.


## 5.3 Save Predictions to csv File

In [ ]:
import csv
with open('submission.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['id','label'])
    for i, phoneme in enumerate(predictions):
        id = i
        label = phoneme
        csv_out.writerow((id, phoneme))

## 5.4 Submit Predictions

In [ ]:
!kaggle competitions submit -c 11785-fall2021-hw3p2 -f submission.csv -m "Please work :)"

100% 309k/309k [00:00<00:00, 1.78MB/s]
Successfully submitted to 11785 Homework 3 Part 2: Seq to Seq